# Computer Vision Mini Exercises

This notebook guides you through a set of mini exercises that demonstrate common computer vision workflows using both **PyTorch** and **TensorFlow**. Each section includes explanations, runnable code, and short practice prompts that you can use in class to reinforce the concepts.

## Learning Objectives

By the end of this lab, students should be able to:

* Load and inspect an image classification dataset.
* Build and train a simple convolutional neural network (CNN) in PyTorch.
* Build and train an equivalent CNN in TensorFlow/Keras.
* Compare and contrast the two frameworks through small exploratory exercises.

---
## 1. Environment Setup

Run the cell below to import the libraries used throughout this notebook. If you are running on a new environment, ensure that `torch`, `torchvision`, `tensorflow`, and `matplotlib` are installed.

In [ ]:
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

# Reproducibility helpers
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
tf.random.set_seed(SEED)

print(f"PyTorch version: {torch.__version__}")
print(f"TensorFlow version: {tf.__version__}")

---
## 2. Loading and Exploring the Dataset

We will use the **Fashion-MNIST** dataset, which consists of 28x28 grayscale clothing images. Both PyTorch and TensorFlow provide convenient access to this dataset.

> **Teaching tip:** Ask students to describe the normalization transform and explain why we apply it before training.

In [ ]:
# PyTorch dataset and dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST(
    root="./data", train=True, download=True, transform=transform
)
test_dataset = datasets.FashionMNIST(
    root="./data", train=False, download=True, transform=transform
)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

class_names = train_dataset.classes
class_names

In [ ]:
# Visual inspection helper
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for ax in axes.ravel():
    image, label = train_dataset[random.randint(0, len(train_dataset) - 1)]
    ax.imshow(image.squeeze(), cmap="gray")
    ax.set_title(class_names[label])
    ax.axis("off")
plt.tight_layout()
plt.show()

### 🎯 Exercise 1: Augmentation Exploration

Try modifying the `transform` pipeline above to include data augmentation (e.g., random rotations or flips). Discuss how augmentation might affect model generalization.

---
## 3. PyTorch Model

Below is a lightweight CNN implemented in PyTorch. It trains quickly on CPU and is intentionally simple so that students can focus on understanding the workflow.

> **Teaching prompt:** Pause after the model definition and ask learners to predict how many parameters are in each layer.

In [ ]:
class TorchCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model_torch = TorchCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_torch.parameters(), lr=1e-3)
model_torch

In [ ]:
def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total


def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

In [ ]:
EPOCHS = 3
for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model_torch, train_loader, optimizer, criterion)
    test_loss, test_acc = evaluate(model_torch, test_loader, criterion)
    print(f"Epoch {epoch + 1}: train_loss={train_loss:.4f}, train_acc={train_acc:.3f}, test_acc={test_acc:.3f}")

### 🎯 Exercise 2: Optimizer Swap

Ask students to replace the Adam optimizer with SGD (optionally with momentum) and observe how the training dynamics change. How does the convergence speed compare?

---
## 4. TensorFlow / Keras Model

We will now build a nearly identical architecture in TensorFlow using the Keras API. Highlight the parallels between the two frameworks and encourage students to identify similarities in layer configuration and training loops.

In [ ]:
# Prepare TensorFlow datasets
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Normalize and add channel dimension
x_train = (x_train / 255.0).astype("float32")[..., np.newaxis]
x_test = (x_test / 255.0).astype("float32")[..., np.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

label_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
tf_model = keras.Sequential([
    keras.layers.Conv2D(32, 3, padding="same", activation="relu", input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(10)
])

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
tf_model.compile(optimizer=keras.optimizers.Adam(1e-3),
                 loss=loss_fn,
                 metrics=["accuracy"])

tf_model.summary()

In [ ]:
history = tf_model.fit(train_ds, validation_data=test_ds, epochs=3)

### 🎯 Exercise 3: Learning Rate Tuning

Have learners adjust the learning rate in the `Adam` optimizer above. Encourage them to compare the resulting accuracy curves with the PyTorch model. Which framework made it easier to track metrics?

---
## 5. Comparing Predictions

Finally, let's compare the predictions from both models on the same set of images. This helps students verify that similar architectures in different frameworks can achieve comparable performance.

In [ ]:
def plot_model_predictions(images, labels, torch_model, keras_model, n_samples=5):
    torch_model.eval()
    idxs = np.random.choice(len(images), size=n_samples, replace=False)

    fig, axes = plt.subplots(n_samples, 3, figsize=(10, 2 * n_samples))
    for row, idx in enumerate(idxs):
        image = images[idx]
        label = labels[idx]
        axes[row, 0].imshow(image.squeeze(), cmap="gray")
        axes[row, 0].set_title(f"True: {label_names[label]}")
        axes[row, 0].axis("off")

        with torch.no_grad():
            torch_input = torch.tensor(image.transpose(2, 0, 1)).unsqueeze(0)
            torch_logits = torch_model(torch_input)
            torch_pred = torch_logits.argmax(dim=1).item()
        axes[row, 1].barh(label_names, torch_logits.squeeze().softmax(dim=0).numpy())
        axes[row, 1].set_title(f"PyTorch Pred: {label_names[torch_pred]}")

        tf_logits = keras_model.predict(image[np.newaxis, ...], verbose=0)
        tf_pred = tf_logits.argmax(axis=1)[0]
        axes[row, 2].barh(label_names, tf.nn.softmax(tf_logits).numpy()[0])
        axes[row, 2].set_title(f"TensorFlow Pred: {label_names[tf_pred]}")

    plt.tight_layout()
    plt.show()

plot_model_predictions(x_test, y_test, model_torch, tf_model)

### 🎯 Exercise 4: Robustness Check

Challenge students to craft a simple perturbation (e.g., adding noise) and evaluate how each model's predictions change. Which model is more robust to the perturbation?

---
## 6. Wrap-Up Discussion

* Summarize the similarities and differences between PyTorch and TensorFlow workflows.
* Highlight how data input pipelines differ across frameworks.
* Encourage students to consider when they might choose one framework over the other.